In [ ]:
## %reset -f
# Required imports
import sys
import proteus
from proteus.iproteus import * 
from proteus import Context, default_n, default_s, default_so
from proteus import Comm
Profiling.logLevel=5
Profiling.verbose=False

import tank
reload(tank)
import tank_so
reload(tank_so)

import numpy as np


from ipywidgets import *
import ipympl
import matplotlib.pyplot as plt
from helpers import *
from IPython.display import clear_output, display

from tables import  openFile
import PostProcessing 
reload(PostProcessing)

from petsc4py import PETSc
from threading import Thread
global simulation_thread, ns
import utilities
reload(utilities)

from IPython.core.display import HTML
from IPython.display import FileLink,FileLinks
import webbrowser

plt.close("all")
# Function to modify tank dimensions and initial SDF using interactive widget
def on_value_change(change):
    Length=tankLength.value
    Height=tankHeight.value
    obsW=obsWidth.value
    obsH=obsHeight.value
    waterL=waterLength.value
    waterH=waterHeight.value
    tank.domain.vertices[-3]=(Length,0.0)
    tank.domain.vertices[-2]=(Length,Height)
    tank.domain.vertices[-1]=(0.0,Height)
    tank.domain.vertices[1]=(obsW[0],0.0)
    tank.domain.vertices[2]=(obsW[0],obsH)
    tank.domain.vertices[3]=(obsW[1],obsH)
    tank.domain.vertices[4]=(obsW[1],0.0)
    name=tank.domain.polyfile
    tank.domain.polyfile=None
    tank.domain.writePoly(name)
    tank.domain.getBoundingBox()
    
    f1.clear()
    plot_domain(tank.domain)
    f1.canvas.draw()
    
    #Water
    tank.waterLevel_x=waterL
    tank.waterLevel_y=waterH
    domain =tank.domain
    sdf = tank.signedDistance
    xg = np.linspace(0, domain.L[0], 20)
    yg = np.linspace(0, domain.L[1], 20)
    xi, yi = np.meshgrid(xg,yg)
    phi = np.zeros(xi.shape)
    for i in range(20):
        for  j in range(20):
            phi[i,j]  = sdf([xg[j],yg[i]],waterL,waterH)
    plt.contourf(xg,yg,phi)
    plt.colorbar()
    plt.contour(xg,yg,phi,levels=[0],linewidths=[5], colors='k')
    plt.title('Bathmetry and Initial SDF')
    plt.axis([0.0, domain.L[0], 0, domain.L[1]]) 
    f1.canvas.draw()
    
# Create Tank Dimension Widgets   

tankLength=FloatSlider(min=0.0, max=3.0, step=0.1, continuous_update=False,value=tank.L[0],
                  description='Tank Length:')
tankHeight=FloatSlider(min=0.1, max=3.0, step=0.01, continuous_update=False,value=tank.L[1],
                  description='Tank Height:')
obsWidth=widgets.FloatRangeSlider(
                                value=[tank.obst_x_start, tank.obst_x_end],min=0.0, max=3.0, step=0.1,
                                description='Obs Range:',
                                continuous_update=False,
                                slider_color='white',
                                color='black'
                            )
obsHeight=FloatSlider(min=0.1, max=3.0, step=0.1, continuous_update=False,value=tank.obst_portions[1],
                  description='Obs Height')
waterLength=FloatSlider(min=0.0, max=0.6, step=0.1, continuous_update=False,value=tank.waterLine_x,
                      description='Water Length:')
waterHeight=FloatSlider(min=0.1, max=0.6, step=0.01, continuous_update=False,value=tank.waterLine_z,
                      description='Water Height:')    


# 
tankLength.observe(on_value_change, names='value')
tankHeight.observe(on_value_change, names='value')
obsWidth.observe(on_value_change, names='value')
obsHeight.observe(on_value_change, names='value')
waterLength.observe(on_value_change, names='value')
waterHeight.observe(on_value_change, names='value')
#
#
#page1=interactive(on_value_change,
#                  Length=tankLength,
#                  Height=tankHeight,
#                  obsW=obsWidth,
#                  obsH=obsHeight,
#                  waterL=waterLength,
#                  waterH=waterHeight)  

tankBox=VBox([tankLength,tankHeight,])
obsBox=VBox([obsWidth,obsHeight])
waterBox=VBox([waterLength,waterHeight])

f1 = plt.figure()
plt.plot([0, 1, 2, 2])
page1Controls=HBox([tankBox,obsBox,waterBox])
page1=VBox([page1Controls,f1.canvas])
on_value_change(True)

# Link widget values with the current tank dimension values
mylink1 = jslink((tankLength, 'value'), (obsWidth, 'max'))
mylink2 = jslink((tankHeight, 'value'), (obsHeight, 'max'))
mylink3 = jslink((tankLength, 'value'), (waterLength, 'max'))
mylink4 = jslink((tankHeight, 'value'), (waterHeight, 'max'))


#============================================================================================================================
#Physics and Numerics
#Load the modules the define the equations to be solved and the numerical methods to use.
#============================================================================================================================


# Define limits for he max min
from math import sqrt
nn=20e3 #Default value for 1 core
he_min=sqrt(tank.domain.L[0]*tank.domain.L[1]/nn)
he_max=tank.domain.vertices[2][1]/2
he_avg=0.5*(he_min+he_max)

#Prepare Widgets
advance=widgets.Checkbox(value=False, description='Expert Options', disabled=False)
showTriangles=widgets.Checkbox(value=True, description='Show Triangles', disabled=False)
dtInit=BoundedFloatText(value=tank.dt_init,min=.001,max=1e1,step=.001,
                        description='dt initial',disabled=True,color='grey')
he=BoundedFloatText(value=tank.he, min=he_min, max=he_max, step=1e-15,
                    description='he', disabled=True, color='grey')
dtFixed=BoundedFloatText(value=tank.dt_fixed/2,min=.001,max=1e1,step=.001,
                       description='dt')
tf=FloatText(value=tank.T,step=.001,
                        description='Final Time')

# Observe checkbox to enable or disable advance options
def handle_advance_change(change):
    dtInit.disabled=not(change.new)
    he.disabled=not(change.new)
    if change.new:
        dtInit.color='black'
        he.color='black'
    else:
        dtInit.color='grey'
        he.color='grey'
        
advance.observe(handle_advance_change, names='value')
btn_parameters = widgets.Button(description="Run Simulation")


def on_button_clicked(b):
    clear_output()
    #mesh refinment
    global simulation_thread, ns
    print("Computing Parameters...")
    tank.triangleOptions="VApq30Dena%8.8f" % ((tank.he**2)/2.0,)
    #
    so = tank_so
    tank.T=tf.value
    tank.dt_fixed=dtFixed.value
    tank.nDTout=int(round(tank.T/tank.dt_fixed))
    tank.he = he.value
    tank.dt_init=dtInit.value
    so.tnList = [0.0,tank.dt_init]+[i*tank.dt_fixed for i in range(1,tank.nDTout+1)]
    info = open("TimeList.txt","w")
    for time in so.tnList:
        info.write(str(time)+"\n")
    info.close()
    tank_so.tnList=so.tnList
    
    pList=[]
    nList=[]
    so.sList=[]
    so.tank=tank
    OptDB = PETSc.Options()
    for (p,n) in so.pnList:
        so.sList.append(default_s)
        pList.append(__import__(p))
        reload(pList[-1])
        nList.append(__import__(n))
        reload(nList[-1])
        pList[-1].name = p
        nList[-1].multilevelLinearSolver = default_n.KSP_petsc4py
        nList[-1].levelLinearSolver = default_n.KSP_petsc4py
        OptDB.setValue(nList[-1].linear_solver_options_prefix+"ksp_type", "preonly")
        OptDB.setValue(nList[-1].linear_solver_options_prefix+"pc_type", "lu")
        OptDB.setValue(nList[-1].linear_solver_options_prefix+"pc_factor_mat_solver_package","superlu_dist")
    opts.save_dof = True
    opts.dataDir='.'
    opts.probDir='.'
    opts.logLevel=7
    opts.verbose=True
    #Temporal Fix meanwhile a tag is added to each simulation
    !rm *.h5
    !rm *.xmf
    #-----------------------------------------------------------------------
    #Numerical Solution Object
    # Now we create an the numerical wavetank object and set it up to run in a thread on each engine.
    #-----------------------------------------------------------------------
    print("Creating Numerical Solution Object...")
    import time
    t1=time.time()
    ns = NumericalSolution.NS_base(so, pList, nList, so.sList, opts)
    # Create a thread wrapper for the simulation.  The target must be an argument-less
    # function so we wrap the call to `calculateSolution` in a simple lambda:
    print("Creating Simulation Thread...")
    simulation_thread = Thread(target = lambda : ns.calculateSolution('run1'))
    print("Starting Thread and Simulation Monitor")
    simulation_thread.start()
    f2.clear()
    plt.plot([])
    f2.canvas.draw()
    utilities.simulation_thread=simulation_thread
    utilities.ns=ns
    utilities.plt=plt
    utilities.f=plt.figure(num=2)
    utilities.tank=tank
    utilities.showTriangles=showTriangles.value   
    utilities.monitor_simulation(refresh=5.0)

    print("Complete in:")
    t2=time.time()
    print(t2-t1)


btn_parameters.on_click(on_button_clicked)
f2=plt.figure()
f2.clear()
plt.plot([])
f2.canvas.draw()
page3=HBox([VBox([tf,dtFixed,advance,he,dtInit,showTriangles,btn_parameters]),f2.canvas])

#============================================================================================================================
# Create Post-Processing
#============================================================================================================================

f3=plt.figure()
f3.clear()
plt.plot([])
f3.canvas.draw()

btn_createImages=Button(description='Create Images')

def on_btn_createImages_clicked(b):
    clear_output()
    print("Creating Images ...")
    f3.clear()
    !rm phi*png
    PostProcessing.tank=tank
    PostProcessing.plt=plt
    PostProcessing.f=plt.figure(num=3)
    PostProcessing.tank_so=tank_so
    PostProcessing.Create_Images()
    clear_output()
    f3.canvas.draw()
    print('Images Created')

btn_createImages.on_click(on_btn_createImages_clicked)
btn_createVideo=Button(description='Create Video',)
    
def on_btn_createVideo_clicked(b):
    clear_output()
    !rm -f tankPhi.mp4; LD_LIBRARY_PATH='' avconv -i phi%4d.png -vcodec libx264 tankPhi.mp4 -loglevel quiet
    PostProcessing.Create_Video()
    clear_output()
    print("Click link to see video:")
    display(FileLink('tankPhi.mp4'))
    
btn_createVideo.on_click(on_btn_createVideo_clicked)

page4=HBox([VBox([btn_createImages,btn_createVideo]),f3.canvas])


#============================================================================================================================
# Widget Display
#============================================================================================================================

# Display Widgets in Tabs
lst=['Bathmetry','Simulation','Post Processing']
children = [page1,page3,page4]
tab = widgets.Tab(children=children)
[tab.set_title(num, name) for num, name in enumerate(lst)]
display(tab)



Computing Parameters...
Creating Numerical Solution Object...
Creating Simulation Thread...
Starting Thread and Simulation Monitor
Monitored for: 0:00:07.527784.
Monitored for: 0:00:15.082308.
Monitored for: 0:00:22.745980.
Monitored for: 0:00:30.637122.
Monitored for: 0:00:38.546324.
Simulation completed!
Monitored for: 0:00:43.563309.
Complete in:
45.0579071045


Mathematical Model
===============

To model a tank with breaking and overtopping, we solve the following air/water flow equations in 2D (a vertical slice):

$$
\begin{eqnarray}
\nabla \cdot \left( \epsilon \mathbf v \right) &=& s \\
\frac{\partial \left( \epsilon \mathbf v \right)}{\partial t} +\nabla \cdot  \left(\epsilon \mathbf v \otimes \mathbf v\right) -  \nabla \cdot \left\{\epsilon \nu_t \left(\nabla  \mathbf v+\nabla \mathbf v^T\right) \right\} + \mathbf r + \frac{\epsilon \nabla p}{\rho} &=& 0 \\
\frac{\partial \phi}{\partial t} + \mathbf v \cdot \nabla \phi &=& 0 \\
\|\nabla \phi\| &=& 1 \\
\frac{\partial \theta}{\partial t} + \nabla \cdot \left( \theta \epsilon \mathbf v \right) &=& 0 \\
H(\phi) &=& \theta 
\end{eqnarray}
$$

Notes about this demo
================

- This software can be run locally on mac, windows, and linux if the Proteus stack is installed.
- It can also be accessed from a cloud service like JupyterHub, Sage Math Cloud, Wakari, or an internal cloud server if we invest in setting one up (you are probably viewing on try.proteustoolkit.org)
- This particular notebook can also be saved as a Python script and run on HPC machines if they have the Proteus software stack installed
- Grid generation, specification of input parameters like waves, boundary conditions, and material properties can all be viewed, set from pre-existing definitions, or redefined in this notebook.
- The simulation is kicked off in the notebook as a seperate thread and monitored
- The data is archived  in XDMF, which can be opened upon completion of the simulation for post-processing, furthermore additional simulations and multiple scripted runs can be orchestrated by modifying this notebook.



Load Proteus
==========

Proteus is a Python package consiting of multiple modules. We are going to define a problem and run a simulation interactively, so we pull in the iproteus (interactive proteus) module to set up a basic environment.

Notes:
---------

- Proteus runs with logging that records verying amounts of information by setting logLevel from 1 to 11
- The log is stored  in a .log file, which can be downloaded
- If Profiling.verbose is set  to True then logging will show up in certain output cells of the notebook, which is usually not what you want

Define the flume geometry
=========================

We will just import a predefined geometry from `tank.py` and modify it with widgets. The geometry includes the tank dimensions and an obstacle


Now we define the tank dimensions (Default: Length=38m, Height=0.6m) using the sliders or the numerical textbox at the right.

Numerical Solution Object
====================

No we create an the numerical wavetank object and set it up to run in a thread on each engine.

In [ ]:
#ns = NumericalSolution.NS_base(so, pList, nList, so.sList, opts)

In [ ]:
#from threading import Thread
## Create a thread wrapper for the simulation.  The target must be an argument-less
## function so we wrap the call to `calculateSolution` in a simple lambda:
#global simulation_thread
#simulation_thread= Thread(target = lambda : ns.calculateSolution('run1'))
print(simulation_thread)

Run the tank
==========

In [ ]:
simulation_thread.start()

In [ ]:
import utilities
fig = plt.figure()
utilities.simulation_thread=simulation_thread
utilities.ns=ns
utilities.plt=plt
utilities.fig=fig
from utilities import *
monitor_simulation(refresh=5.0)

Post-process the numerical solution
==========================

In [ ]:
triangles=utilities.triangles
x=utilities.x
y=utilities.y
print(triangles)
print(x)
print(y)

In [ ]:
#import numpy as  np
#x = np.concatenate(view['x'])
#y = np.concatenate(view['y'])
#shifts = np.cumsum([0]+view['nn'][:-1])
#flat_triangles = np.concatenate([ tri + shift for tri,shift in zip(view['triangles'], shifts) ])
plt.figure()
plt.triplot(x,y,triangles)

In [ ]:
!rm phi*png

In [ ]:
from tables import  openFile
archive = openFile('tank_p.h5','r')
import matplotlib.tri as mtri
domain = tank.domain
nodes = archive.getNode("/nodesSpatial_Domain0")
x=nodes[:,0]
y=nodes[:,1]
elements = archive.getNode("/elementsSpatial_Domain0")
triang = mtri.Triangulation(x, y, elements)
xg = np.linspace(0, domain.L[0], 20)
yg = np.linspace(0, domain.L[1], 20)
xi, yi = np.meshgrid(xg,yg)
plt.figure()
for it,t in enumerate(tank_so.tnList[2:]):
    phi = archive.getNode("/phi_t"+`it`)
    vof = archive.getNode("/vof_t"+`it`)
    wvof = np.ones(vof.shape,'d')
    wvof -= vof
    u = archive.getNode("/u_t"+`it`)
    v = archive.getNode("/v_t"+`it`)
    plt.clf()
    plt.xlabel(r'z[m]')
    plt.ylabel(r'x[m]')
    colors = ['b','g','r','c','m','y','k','w']
    plt.xlim(domain.x[0]-0.1*domain.L[0],domain.x[0]+domain.L[0]+0.1*domain.L[0])    
    for si,s in enumerate(domain.segments):
        plt.plot([domain.vertices[s[0]][0],
                     domain.vertices[s[1]][0]],
                    [domain.vertices[s[0]][1],
                     domain.vertices[s[1]][1]],
                    color=colors[domain.segmentFlags[si]-1],
                    linewidth=2,
                    marker='o')
    plt.tricontourf(x,y,elements,wvof*np.sqrt(u[:]**2 + v[:]**2))
    plt.tricontour(x,y,elements,phi,[0], linewidth=4)
    u_interp_lin = mtri.LinearTriInterpolator(triang, u[:])
    v_interp_lin = mtri.LinearTriInterpolator(triang, v[:])
    u_lin = u_interp_lin(xi, yi)
    v_lin = v_interp_lin(xi, yi)
    plt.streamplot(xg, yg, u_lin, v_lin,color='k')
    plt.title('T=%2.2f' % (t,))
    plt.axis('equal')
    plt.xlim((0,domain.L[0]))
    plt.savefig('phi%4.4d.png' % (it,))

In [ ]:
!rm tankPhi.mp4; avconv -i phi%4d.png -vcodec libx264 tankPhi.mp4

In [ ]:
from IPython.core.display import HTML
data_uri_mp4 = open("tankPhi.mp4", "rb").read().encode("base64").replace("\n", "")
video_tag = """<video controls>
<source type ="video/mp4" src="data:video/mp4;base64,{mp4}"/>
Your browser does not support the video tag
</video>""".format(mp4=data_uri_mp4)
HTML(data=video_tag)

In [ ]:
from IPython.display import FileLink,FileLinks
FileLink('tankPhi.mp4')